In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Telecom subscriber churn prediction on Vertex AI

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/subscriber_churn_prediction/telecom-subscriber-churn-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/subscriber_churn_prediction/telecom-subscriber-churn-prediction.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png\" alt="Colab logo"> Run in Colab
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/subscriber_churn_prediction/telecom-subscriber-churn-prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

This example demonstrates building a subscriber churn prediction model on a [telecom customer churn dataset](https://www.kaggle.com/c/customer-churn-prediction-2020/overview). The generated churn model is further deployed to Vertex AI Endpoints and explanations are generated using the Explainable AI feature of Vertex AI. 

### Objective

This tutorial shows you how to do exploratory data analysis, preprocess data, train, deploy and get predictions from a churn prediction model on a tabular churn dataset. The objectives of this tutorial are as follows:

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model` resource
- `Vertex AI Endpoint` resource
- `Vertex Explainable AI`
- Google Cloud Storage

The steps performed include:

- Load data from a Cloud Storage path
- Perform exploratory data analysis (EDA)
- Preprocess the data
- Train a scikit-learn model
- Evaluate the scikit-learn model
- Save the model to a Cloud Storage path
- Create a model and an endpoint in Vertex AI
- Deploy the trained model to an endpoint
- Generate predictions and explanations on test data from the hosted model
- Undeploy the model resource

### Dataset

The dataset used in this tutorial is Telecom-Customer Churn dataset publicly available on Kaggle. See [Customer Churn Prediction 2020](https://www.kaggle.com/c/customer-churn-prediction-2020/data). This dataset is used to build and deploy a churn prediction model using Vertex AI in this notebook.

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Installation

Install the following packages required to execute this notebook. 

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"
    
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform \
                                    google-cloud-storage \
                                    category_encoders \
                                    seaborn \
                                    scikit-learn \
                                    pandas \
                                    fsspec \
                                    gcsfs -q 

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. It is recommended that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. 

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**


When you create a model in Vertex AI using the Cloud SDK, you give a Cloud Storage path where the trained model is saved. 
In this tutorial, Vertex AI saves the trained model to a Cloud Storage bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

## Tutorial

### Import required libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
import pickle
# configure to don't display the warnings
import warnings

import category_encoders as ce
import seaborn as sns
from google.cloud import aiplatform, storage
from google.cloud.aiplatform_v1.types import SampledShapleyAttribution
from google.cloud.aiplatform_v1.types.explanation import ExplanationParameters
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

### Load data from Cloud Storage using Pandas

The Telecom-Customer Churn dataset from [Kaggle](https://www.kaggle.com/c/customer-churn-prediction-2020/overview) is made available on a public Cloud Storage bucket at: 

```gs://cloud-samples-data/vertex-ai/managed_notebooks/telecom_churn_prediction/train.csv```

Use Pandas to read data directly from the URI.

In [ ]:
df = pd.read_csv(
    "gs://cloud-samples-data/vertex-ai/managed_notebooks/telecom_churn_prediction/train.csv"
)
print(df.shape)
df.head()

## Perform EDA


Check the data types and null counts of the fields.

In [ ]:
df.info()

The current dataset doesn't have any null or empty fields in it.

Check the class imbalance.

In [ ]:
df["churn"].value_counts(normalize=True)

There are 14% churners in the data which is not bad for training a churn prediction model. If the class imbalance seems high, oversampling or undersampling techniques can be considered to balance the class distribution.

Separate the caetgorical and numerical columns.

In [ ]:
categ_cols = ["state", "area_code", "international_plan", "voice_mail_plan"]
target = "churn"
num_cols = [i for i in df.columns if i not in categ_cols and i != target]
print(len(categ_cols), len(num_cols))

Plot the level distribution for the categorical columns.

In [ ]:
for i in categ_cols:
    df[i].value_counts().plot(kind="bar")
    plt.title(i)
    plt.show()

In [ ]:
print(num_cols)
df["number_vmail_messages"].describe()

Check the distributions for the numerical columns.

In [ ]:
for i in num_cols:
    # check the Price field's distribution
    _, ax = plt.subplots(1, 2, figsize=(10, 4))
    df[i].plot(kind="box", ax=ax[0])
    df[i].plot(kind="hist", ax=ax[1])
    plt.title(i)
    plt.show()

In [ ]:
# check pairplots for selected features
selected_features = [
    "total_day_calls",
    "total_eve_calls",
    "number_customer_service_calls",
    "number_vmail_messages",
    "account_length",
    "total_day_charge",
    "total_eve_charge",
]
sns.pairplot(df[selected_features])
plt.show()

Plot a heat map of the correlation matrix for the numerical features.

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df[num_cols].corr(), annot=True)
plt.show()

### Observations from EDA

- There are many levels/categories in the categorical field <b>state</b>. In further steps, creating one-hot encoding vectors for this field would increase the columns drastically and so a binary encoding technique will be considered for encoding this field.
- There are only 9% of customers in the data who have had international plans.
- There are only a few customers who make frequent calls to customer service.
- Only 25% of the customers had at least 16 voicemail messages and thus there was skewness in the distribution of the `number_vmail_messages` field.
- Most of the feature combinations in the pair plot show a circular pattern that suggests that there is almost no correlation between the corresponding two features.
- There seems to be a high correlation between minutes and charge. Either one of them can be dropped to avoid multi-collinearity or redundant features in the data.

### Preprocess the data

Drop the fields corresponding to the highly-correlated features.

In [ ]:
drop_cols = [
    "total_day_charge",
    "total_eve_charge",
    "total_night_charge",
    "total_intl_charge",
]
df.drop(columns=drop_cols, inplace=True)
num_cols = list(set(num_cols).difference(set(drop_cols)))
df.shape

Binary encode the state feature (as there are many levels/categories).

In [ ]:
encoder = ce.BinaryEncoder(cols=["state"], return_df=True)
data_encoded = encoder.fit_transform(df)
data_encoded.head()

One-hot encode (drop the first level-column to avoid dummy-variable trap scenarios) the remaining categorical variables.

In [ ]:
def encode_cols(data, col):
    # Creating a dummy variable for the variable 'CategoryID' and dropping the first one.
    categ = pd.get_dummies(data[col], prefix=col, drop_first=True)
    # Adding the results to the master dataframe
    data = pd.concat([data, categ], axis=1)
    return data


for i in categ_cols + [target]:
    if i != "state":
        data_encoded = encode_cols(data_encoded, i)
        data_encoded.drop(columns=[i], inplace=True)

data_encoded.shape

Check the data.

In [ ]:
data_encoded.head()

Check the columns.

In [ ]:
data_encoded.columns

Split the data into train and test sets.

In [ ]:
X = data_encoded[[i for i in data_encoded.columns if i not in ["churn_yes"]]].copy()
y = data_encoded["churn_yes"].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=100
)
print(X_train.shape, X_test.shape)

Scale the numerical data using `MinMaxScaler`.

In [ ]:
sc = MinMaxScaler()
X_train.loc[:, num_cols] = sc.fit_transform(X_train[num_cols])
X_test.loc[:, num_cols] = sc.transform(X_test[num_cols])

## Train a logistic regression model using scikit-learn


The argument `class_weight` adjusts the class weights to the target feature.

In [ ]:
model = LogisticRegression(class_weight="balanced")
model = model.fit(X_train, y_train)

## Evaluate the trained model


#### Plot the ROC and show AUC on train and test sets

Plot the ROC for the model on train data.

In [ ]:
plot_roc_curve(model, X_train, y_train, drop_intermediate=False)
plt.show()

# plot the ROC for the model on test data
plot_roc_curve(model, X_test, y_test, drop_intermediate=False)
plt.show()

#### Determine the optimal threshold for the binary classification 

In general, the logistic regression model outputs probability scores between 0 and 1 and a threshold needs to be determined to assign a class label. Depending on the sensitivity (true-positive rate) and specificity (true-negative rate) of the model, an optimal threshold can be determined.

Create columns with 10 different probability cutoffs.

In [ ]:
y_train_pred = model.predict_proba(X_train)[:, 1]
numbers = [float(x) / 10 for x in range(10)]
y_train_pred_df = pd.DataFrame({"true": y_train, "pred": y_train_pred})
for i in numbers:
    y_train_pred_df[i] = y_train_pred_df.pred.map(lambda x: 1 if x > i else 0)

Now calculate accuracy, sensitivity, and specificity for various probability cutoffs.

In [ ]:
cutoff_df = pd.DataFrame(columns=["prob", "accuracy", "sensitivity", "specificity"])

# compute the parameters for each threshold considered
for i in numbers:
    cm1 = confusion_matrix(y_train_pred_df.true, y_train_pred_df[i])
    total1 = sum(sum(cm1))
    accuracy = (cm1[0, 0] + cm1[1, 1]) / total1

    speci = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])
    sensi = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
    cutoff_df.loc[i] = [i, accuracy, sensi, speci]

# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x="prob", y=["accuracy", "sensitivity", "specificity"])
plt.title("Comparison of performance across various thresholds")
plt.show()

In general, a model with balanced sensitivity and specificity is preferred. In the current case, the threshold where the sensitivity and specifity curves intersect can be considered an optimal threshold.

In [ ]:
threshold = 0.5

# Evaluate train and test sets
y_test_pred = model.predict_proba(X_test)[:, 1]

# to get the performance stats, lets define a handy function


def print_stats(y_true, y_pred):
    # Confusion matrix

    confusion = confusion_matrix(y_true=y_true, y_pred=y_pred)
    print("Confusion Matrix: ")
    print(confusion)

    TP = confusion[1, 1]  # true positive
    TN = confusion[0, 0]  # true negatives
    FP = confusion[0, 1]  # false positives
    FN = confusion[1, 0]  # false negatives

    # Let's see the sensitivity or recall of our logistic regression model
    sensitivity = TP / float(TP + FN)
    print("sensitivity = ", sensitivity)
    # Let us calculate specificity
    specificity = TN / float(TN + FP)
    print("specificity = ", specificity)
    # Calculate false postive rate - predicting conversion when customer didn't convert
    fpr = FP / float(TN + FP)
    print("False positive rate = ", fpr)
    # positive predictive value
    precision = TP / float(TP + FP)
    print("precision = ", precision)
    # accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    print("accuracy = ", accuracy)
    return


y_train_pred_sm = [1 if i > threshold else 0 for i in y_train_pred]
y_test_pred_sm = [1 if i > threshold else 0 for i in y_test_pred]
# Print the metrics for the model
# on train data
print("Train Data : ")
print_stats(y_train, y_train_pred_sm)
print("\n", "*" * 30, "\n")
# on test data
print("Test Data : ")
print_stats(y_test, y_test_pred_sm)

While the model's sensitivity and specificity are looking decent, the precision can be considered low. This type of situation may be acceptable to some extent because from a business standpoint in the telecom industry, it still makes sense to identify churners even though it means there'd be some mis-classifications of non-churners as churners.

## Save the model to a Cloud Storage path


Save the trained model to a local file `model.pkl`.

In [ ]:
FILE_NAME = "model.pkl"
with open(FILE_NAME, "wb") as file:
    pickle.dump(model, file)

# Upload the saved model file to Cloud Storage
BLOB_PATH = (
    "[your-blob-path]"  # leave blank if no folders inside the bucket are needed.
)

if BLOB_PATH == ("[your-blob-path]"):
    BLOB_PATH = ""

BLOB_NAME = BLOB_PATH + FILE_NAME

bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

## Create a model with Explainable AI support in Vertex AI

Before creating a model, configure the explanations for the model. For further details, see [Configuring explanations in Vertex AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations#scikit-learn-and-xgboost-pre-built-containers).

Set a display name for the model resource.

In [ ]:
# Set the model display name
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "subscriber_churn_model"

ARTIFACT_GCS_PATH = f"gs://{BUCKET_NAME}/{BLOB_PATH}"

# Feature-name(Inp_feature) and Output-name(Model_output) can be arbitrary
exp_metadata = {"inputs": {"Inp_feature": {}}, "outputs": {"Model_output": {}}}

In [ ]:
# Create a Vertex AI model resource with support for explanations


aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
    explanation_metadata=exp_metadata,
    explanation_parameters=ExplanationParameters(
        sampled_shapley_attribution=SampledShapleyAttribution(path_count=25)
    ),
)

model.wait()

print(model.display_name)
print(model.resource_name)

Alternatively, the following `gcloud` command can be used to create the model resource. The `explanation-metadata.json` file consists of the metadata that is used to configure explanations for the model resource.

```
gcloud beta ai models upload \
  --region=$REGION \
  --display-name=$MODEL_DISPLAY_NAME \
  --container-image-uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest" \
  --artifact-uri=$ARTIFACT_GCS_PATH \
  --explanation-method=sampled-shapley \
  --explanation-path-count=25 \
  --explanation-metadata-file=explanation-metadata.json
```

### Create an endpoint

In [ ]:
ENDPOINT_DISPLAY_NAME = "[your-endpoint-display-name]"  # @param {type:"string"}
if ENDPOINT_DISPLAY_NAME == "[your-endpoint-display-name]":
    ENDPOINT_DISPLAY_NAME = "subsc_churn_endpoint"

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME, project=PROJECT_ID, location=REGION
)

print(endpoint.display_name)
print(endpoint.resource_name)

### Deploy the model to the created endpoint

Configure the depoyment name, machine-type, and other parameters for the deployment.

In [ ]:
DEPLOYED_MODEL_NAME = "[deployment-model-name]"  # @param {type:"string"}
MACHINE_TYPE = "n1-standard-4"

if DEPLOYED_MODEL_NAME == "[deployment-model-name]":
    DEPLOYED_MODEL_NAME = "subsc_churn_deployment"

In [ ]:
# deploy the model to the endpoint
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=DEPLOYED_MODEL_NAME,
    machine_type=MACHINE_TYPE,
)

model.wait()

print(model.display_name)
print(model.resource_name)

To ensure the model is deployed, the ID of the deployed model can be checked using the `endpoint.list_models()` method.

In [ ]:
endpoint.list_models()

## Get explanations from the deployed model


Get explanations for a test instance from the hosted model.

In [ ]:
# format a test instance as the request's payload
test_json = [X_test.iloc[0].tolist()]

### Get explanations and plot the feature attributions

In [ ]:
features = X_train.columns.to_list()


def plot_attributions(attrs):
    """
    Function to plot the features and their attributions for an instance
    """
    rows = {"feature_name": [], "attribution": []}
    for i, val in enumerate(features):
        rows["feature_name"].append(val)
        rows["attribution"].append(attrs["Inp_feature"][i])
    attr_df = pd.DataFrame(rows).set_index("feature_name")
    attr_df.plot(kind="bar")
    plt.show()
    return


def explain_tabular_sample(project: str, location: str, endpoint, instances: list):
    """
    Function to make an explanation request for the specified payload and generate feature attribution plots
    """
    aiplatform.init(project=project, location=location)

    # endpoint = aiplatform.Endpoint(endpoint_id)

    response = endpoint.explain(instances=instances)
    print("#" * 10 + "Explanations" + "#" * 10)
    for explanation in response.explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions

        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print("   output_index:", output_index)

            plot_attributions(attribution.feature_attributions)

    print("#" * 10 + "Predictions" + "#" * 10)
    for prediction in response.predictions:
        print(prediction)

    return response


# Get explanations for the test instance
prediction = explain_tabular_sample(PROJECT_ID, REGION, endpoint, test_json)

## Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
* Vertex AI Model
* Vertex AI Endpoint
* Cloud Storage bucket

Set `delete_bucket` to *True* to delete the Cloud Storage bucket.

In [ ]:
# Undeploy model
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the Cloud Storage bucket
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI